# NMEG SPEI

Calculate SPEI (Standardized precipitation-evaporation index) for NMEG data and historical SPEI at the sites.

## First calculate SPEI for NMEG data period

In [1]:
# Load daily data files
source('../r_functions/load_nmeg.r')

seg <- daily_to_xts(get_daily_file('Seg', 'aflx', make_new=FALSE))
ses <- daily_to_xts(get_daily_file('Ses', 'aflx', make_new=FALSE))
wjs <- daily_to_xts(get_daily_file('Wjs', 'aflx', make_new=FALSE))
mpj <- daily_to_xts(get_daily_file('Mpj', 'aflx', make_new=FALSE))
vcp <- daily_to_xts(get_daily_file('Vcp', 'aflx', make_new=FALSE))
vcm <- daily_to_xts(get_daily_file('Vcm', 'aflx', make_new=FALSE))

head(vcm)

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



            GPP_g_int RECO_g_int FC_F_g_int ET_mm_fullday_0  P_F       TA_F
2007-01-01 0.12321924  0.2005312 0.07731197       1.0004046 0.00 -2.8387670
2007-01-02 0.09572404  0.4381529 0.34242888       0.9419793 2.54 -1.4170800
2007-01-03 0.22328216  0.3979440 0.17466182       0.7102144 0.00  0.8552379
2007-01-04 0.29385697  0.4046708 0.11081380       0.4587443 0.00  1.0826423
2007-01-05 0.19983311  0.2330692 0.03323612       0.4499571 2.54 -6.3807404
2007-01-06 0.05094230  0.4979468 0.44700450       1.7626831 0.00 -9.9878025
               RH_F   SW_IN_F RNET      VPD_F  TA_F_min VPD_F_min TA_F_max
2007-01-01 49.87072 101.64726   NA 0.25164502  -5.15918  0.115820 -0.75936
2007-01-02 40.51016 115.64632   NA 0.33900065  -4.92205  0.114977  1.98509
2007-01-03 32.04234 128.64500   NA 0.44169131  -1.83173  0.288466  5.06760
2007-01-04 41.48273 131.89137   NA 0.39319227  -1.77366  0.237693  4.15514
2007-01-05 81.55373  21.52585   NA 0.07710852 -10.23000  0.016431 -2.01179
2007-01-06 67.6612

## Examine climatic water deficit for 4 sites

In [2]:
# Calculate climatic water deficit
sitelist <- list( seg, ses, wjs, mpj, vcp, vcm )
sitenames <- list( 'Seg', 'Ses', 'Wjs', 'Mpj', 'Vcp', 'Vcm' )
cwdiff <- data.frame()
for (i in 1:6){
    cwdiff <- cbind( cwdiff, apply.monthly(sitelist[[i]]$P_F, FUN=sum) - 
                    apply.monthly(sitelist[[i]]$PET_F_mm_daytime, FUN=sum))
}
colnames(cwdiff) <- sitenames
tail(cwdiff)
#plot.xts(cwdiff, screens=1)


                   Seg        Ses        Wjs         Mpj         Vcp        Vcm
2014-07-31 -12.8465358 -35.115332  17.166474  29.7486179  -3.0439323  58.294678
2014-08-31 -37.0154938 -37.451062 -70.165140 -72.3479162 -76.5392942  14.174822
2014-09-30  -0.7954797 -12.892907  21.870188 -10.3604224 -64.1132265 -41.763136
2014-10-31 -23.2139050 -17.646974 -46.321232 -35.2171479 -20.1961291  -1.743432
2014-11-30  -4.0476426  -5.131684  -1.966827  -0.6147046   0.7675888   8.837995
2014-12-31   1.5877486  -1.766598  -5.781882   7.8099529  62.7040691  19.357315

## Load packages for calculating SPEI
This package is on CRAN [here](https://cran.r-project.org/web/packages/SPEI/index.html). Website for the project is <http://sac.csic.es/spei/index.html>

If not installed use "install.packages('SPEI')"

In [3]:
library('SPEI')
library('xts')

Loading required package: lmomco
Loading required package: parallel
# Package SPEI (1.6) loaded [try SPEINews()].


In [37]:
# Function for retrieving SPEI values from a dataframe at
# a number of different timesteps
get_spei_steps <- function(df, sitename, tstep='monthly',
                           int_steps=seq(1, 12), plot=TRUE){
    if (tstep=='monthly'){
        freq <- 12
        } else if (tstep=='weekly'){
        freq <- 52
    }
    # Get start date
    startmon <- as.numeric(as.yearmon(index(df[1])))
    startyr <- floor(startmon)
    # Calculate climatic water difference on a weekly or monthly scale
    if (tstep=='monthly'){
        cwdiff <- apply.monthly(df$P_F, FUN=sum) - apply.monthly(df$PET_F_mm_daytime, FUN=sum)
    } else if (tstep=='weekly'){
        cwdiff <- apply.weekly(df$P_F, FUN=sum) - apply.weekly(df$PET_F_mm_daytime, FUN=sum)
    }
    colnames(cwdiff) <- sitename
    for (i in 1:length(int_steps)){
        # Integration period for SPEI (in number of timesteps)
        SPEI_int_per <- int_steps[i]
        # Get spei for that integration period
        spei_int <- spei(ts(cwdiff, frequency=freq, start=c(startyr, 1)), SPEI_int_per, na.rm=TRUE)
        if (plot){
            plot(spei_int)
        }
        # Extract the spei values from the returned object and make xts
        spei_int <- xts(as.vector(spei_int$fitted),  index(cwdiff))
        colnames(spei_int) <- paste('SPEI_', tstep, '_', as.character(SPEI_int_per), sep='')
        if (i==1){
            df_new <- spei_int
            print(head(df_new))
        } else if (i > 1) {
            df_new <- cbind(df_new, spei_int)
        }
    }
    
    # Somehow a funky entry at 2009-2-12 is created, remove
    #print(nrow(df_new))
    #df_new <- df_new[c('::2009-01-31', '2009-02-28::')]
    #print(nrow(df_new))
    
    # There may be both infinite and NA values in the output
    # Convert -Inf to NA
    print(sum(is.na(df_new)))
    df_new[!is.finite(df_new)] <- NA
    print(sum(is.na(df_new)))
    
    # Interpolate over NA values
    df_new_interp <- na.approx(df_new)
    #print(head(df_new))
    return( list(df_new, df_new_interp) )
}

In [38]:
# Create monthly files
int_steps1 <- seq(1, 24)
for (j in 1:length(sitelist)){
    df_j <- sitelist[[j]]
    spei_monthly <- get_spei_steps(df_j, 'Ses', int_steps=int_steps1, tstep='monthly', plot=FALSE)
    outfile <- paste('../processed_data/spei/SPEI_monthly_US-',
                     sitenames[[j]], '.csv', sep='')
    write.zoo(spei_monthly[[1]], file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
    outfile <- paste('../processed_data/spei/SPEI_monthly_US-', sitenames[[j]],
                     '_nainterp.csv', sep='')
    write.zoo(spei_monthly[[2]], file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
}

           SPEI_monthly_1
2007-01-31      0.5726167
2007-02-28     -0.1233249
2007-03-31      1.6786612
2007-04-30      0.4982194
2007-05-31     -1.3626615
2007-06-30     -0.7554115
[1] 282
[1] 282
           SPEI_monthly_1
2007-01-31      0.9152357
2007-02-28      0.5264625
2007-03-31      1.6913303
2007-04-30      1.4465155
2007-05-31      1.4998972
2007-06-30     -0.3966785
[1] 540
[1] 541
           SPEI_monthly_1
2007-01-31             NA
2007-02-28             NA
2007-03-31             NA
2007-04-30             NA
2007-05-31             NA
2007-06-30       1.715981
[1] 376
[1] 377
           SPEI_monthly_1
2007-01-31             NA
2007-02-28             NA
2007-03-31             NA
2007-04-30             NA
2007-05-31             NA
2007-06-30             NA
[1] 330
[1] 330
           SPEI_monthly_1
2007-01-31      0.8187068
2007-02-28      1.4301584
2007-03-31      0.7665514
2007-04-30      0.9527582
2007-05-31      1.7723776
2007-06-30      0.2670940
[1] 558
[1] 558
          

In [39]:
# Now weekly
#
# WARNING - note that weekly SPEI generates more NA values - not sure why
#
int_steps2 <- seq(4, 96, 4)
for (j in 1:length(sitelist)){
    df_j <- sitelist[[j]]
    spei_weekly <- get_spei_steps(df_j, 'Ses', int_steps=int_steps2, tstep='weekly', plot=FALSE)
    outfile <- paste('../processed_data/spei/SPEI_weekly_US-',
                     sitenames[[j]], '.csv', sep='')
    write.zoo(spei_weekly[[1]], file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
    outfile <- paste('../processed_data/spei/SPEI_weekly_US-', sitenames[[j]],
                     '_nainterp.csv', sep='')
    write.zoo(spei_weekly[[2]], file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
}

           SPEI_weekly_4
2007-01-07            NA
2007-01-14            NA
2007-01-21            NA
2007-01-28    0.69534779
2007-02-04    0.12589167
2007-02-11   -0.06255176
[1] 1288
[1] 1293
           SPEI_weekly_4
2007-01-07            NA
2007-01-14            NA
2007-01-21            NA
2007-01-28     0.7348555
2007-02-04     0.6007516
2007-02-11     0.6921900
[1] 2170
[1] 2172
           SPEI_weekly_4
2007-01-07            NA
2007-01-14            NA
2007-01-21            NA
2007-01-28     0.7226340
2007-02-04     0.8383332
2007-02-11     0.9087611
[1] 1783
[1] 1783
           SPEI_weekly_4
2007-01-07            NA
2007-01-14            NA
2007-01-21            NA
2007-01-28     0.2561351
2007-02-04     0.6087056
2007-02-11     0.8052870
[1] 1414
[1] 1420
           SPEI_weekly_4
2007-01-07            NA
2007-01-14            NA
2007-01-21            NA
2007-01-28     0.9851334
2007-02-04     1.2832333
2007-02-11     1.3191022
[1] 1866
[1] 1867
           SPEI_weekly_4
2007-01-07

## Now calculate SPEI for historical periods (PRISM)

In [41]:
fpath <- '/home/greg/sftp/eddyflux/Ancillary_met_data/PRISM_monthly/'
fname_ppt <- 'PRISM_Monthly_ppt_1981_2014.csv'
fname_tmean <-'PRISM_Monthly_tmean_1981_2014.csv'
#header <- read.csv(fname, skip=3, nrows=1)
df_ppt <- read.csv(paste(fpath, fname_ppt, sep=''))
df_tmean <- read.csv(paste(fpath, fname_tmean, sep=''))
df_ppt <- xts( df_ppt[,2:ncol(df_ppt)], as.Date(df_ppt$date))
df_tmean <- xts( df_tmean[,2:ncol(df_tmean)], as.Date(df_tmean$date))
head(df_ppt)
head(df_tmean)

           US.Wjs US.Mpj US.Mpg US.Seg US.Sen US.Ses US.Vcm US.Vcp
1981-01-31   3.88   5.38   5.26   5.38   6.10   5.58   4.21   4.39
1981-02-28   5.97   8.40   8.55   4.73   4.95   4.59  28.38  23.72
1981-03-31  16.98  24.01  24.16  16.08  16.50  16.11 161.74 136.64
1981-04-30  11.00  19.80  19.94  10.30  13.20  10.74  47.23  43.04
1981-05-31  10.12  16.72  15.86   9.75  10.13   9.26  88.62  76.18
1981-06-30  20.76  16.05  15.49  12.04  12.21  10.90  61.35  57.34

           US.Wjs US.Mpj US.Mpg US.Seg US.Sen US.Ses US.Vcm US.Vcp
1981-01-31  2.075  1.340  1.515  3.615  3.535  3.620 -2.020 -0.890
1981-02-28  4.055  3.510  3.665  6.205  6.010  6.320 -2.580 -1.385
1981-03-31  5.245  4.555  4.675  8.080  7.910  8.095 -2.415 -1.115
1981-04-30 12.065 11.015 11.170 15.190 14.915 15.240  3.945  5.420
1981-05-31 14.975 13.525 13.695 18.585 18.305 18.615  6.125  7.640
1981-06-30 21.625 20.955 21.180 24.770 24.505 24.850 13.450 14.710

In [42]:
# Get site coordinates
coords <- read.csv('../site_coords.txt')
head(coords)

,sitecode,lat,lon
1,US-Wjs,34.42549,-105.8615
2,US-Mpj,34.43845,-106.2377
3,US-Mpg,34.44682,-106.2134
4,US-Seg,34.36233,-106.7019
5,US-Sen,34.35802,-106.6799
6,US-Ses,34.33494,-106.7442


## Calculate SPEI using thornwaite PET

In [44]:
plot_spei <- FALSE
int_steps1 <- seq(1, 24)

for (i in 1:length(sitenames)){
    # Get site name and latitude
    sitename1 <- paste('US-', sitenames[[i]], sep='')
    lat <- coords[coords$sitecode==sitename1, 2]
    sitename2 <- paste('US.', sitenames[[i]], sep='') # Format sitename for PRISM datasets
    # Calculate pet and climatic water diff with thornthwaite
    pet <- thornthwaite(df_tmean[,sitename2], lat, na.rm=T)
    cwdiff <- df_ppt[,sitename2] - pet
    for (j in 1:length(int_steps1)) {
        int_per <- int_steps1[j]
        spei_int <- spei(ts(cwdiff, frequency=12, start=c(1981, 1)), int_per, na.rm=TRUE)
        if (plot_spei){
            plot(spei_int)
        }
        spei_int <- xts(as.vector(spei_int$fitted),  index(df_tmean))
        colnames(spei_int) <- paste('SPEI_monthly_', as.character(int_per), sep='')
        if (j==1){
            spei_site <- spei_int
        } else {
            spei_site <- cbind(spei_site, spei_int)
        }
    }
    # There may be both infinite and NA values in the output
    # Convert -Inf to NA
    print(sum(is.na(spei_site)))
    spei_site[!is.finite(spei_site)] <- NA
    print(sum(is.na(spei_site)))
    # Actually there are no NA values - ignore this
    
    # Interpolate over NA values
    #spei_site_interp <- na.approx(spei_site)
    
    # Write files
    outfile <- paste('../processed_data/spei/SPEI_PRISM_monthly_', sitename1,
                     '.csv', sep='')
    write.zoo(spei_site, file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
    
    #outfile <- paste('../processed_data/spei/SPEI_PRISM_monthly_', sitename1,
    #                 '_nainterp.csv', sep='')
    #write.zoo(spei_site_interp, file = outfile,
    #          index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
}

Warning message:
: Incompatible methods ("Ops.xts", "Ops.ts") for "-"

[1] 276
[1] 276


Warning message:
: Incompatible methods ("Ops.xts", "Ops.ts") for "-"

[1] 276
[1] 276


Warning message:
: Incompatible methods ("Ops.xts", "Ops.ts") for "-"

...

[1] 276
[1] 276


Warning message:
: Incompatible methods ("Ops.xts", "Ops.ts") for "-"

[1] 276
[1] 276


Warning message:
: Incompatible methods ("Ops.xts", "Ops.ts") for "-"

[1] 276
[1] 276
